
# Main file to run quantitative phase recovery script
Original script was written in Matlab by Adrien Descloux (see below)
Author: mengelhardt
main.py (was QP_main.m)
This file contains a basic pipeline for QP retrieval from brightfield stacks
    - 3D image stack loading
    - 3D stack preprocessing
    - processing parameters definition
    - phase calculation
    - display of the results
    
---------------------------------------
A detailled description of the theory supporting this program can be found in : 
"Descloux, A., et al. "Combined multi-plane phase retrieval and 
  super-resolution optical fluctuation imaging for 4D cell microscopy." 
  Nature Photonics 12.3 (2018): 165."

  Copyright © 2018 Adrien Descloux - adrien.descloux@epfl.ch, 
  École Polytechnique Fédérale de Lausanne, LBEN/LOB,
  BM 5.134, Station 17, 1015 Lausanne, Switzerland.

 	This program is free software: you can redistribute it and/or modify
 	it under the terms of the GNU General Public License as published by
 	the Free Software Foundation, either version 3 of the License, or
 	(at your option) any later version.

 	This program is distributed in the hope that it will be useful,
 	but WITHOUT ANY WARRANTY; without even the implied warranty of
 	MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 	GNU General Public License for more details.

 	You should have received a copy of the GNU General Public License
 	along with this program.  If not, see <http://www.gnu.org/licenses/>.


# load modules

In [3]:
import numpy as np
import os
#import napari
#from utils.io import writeData
from utils.cropXY import cropXY
from utils.cropCoregMask import cropCoregMask
from utils.phase_structure import phase_structure
from utils.getQP import getQP
#from utils.plotStack import plotStack
import tifffile as tf
from skimage import io
from tqdm import tqdm

# load data

In [ ]:
path = r"C:\Users\mengelhardt\data\local\20220922\cos7_p9_FOV__7\registered"
filename = os.listdir(path)
fname = [os.path.join(path, x) for x in filename if "." in x] #. as indicator for file vs folder 
idx = 0
stack = io.imread(os.path.join(path, fname[idx]))

In [ ]:
#%% 3D image stack loading
#path = r"C:\Users\mengelhardt\data\local\20220922\phase_calibration\1um_FOV1\stack_2\cropped\crop"

#stack, fname = loadData(path) # loadData also takes filepath as input 
# flag if you want to explore the data in napari
PLOT_FLAG = False 
print(stack.shape)

## assign dimensions accordingly
permute if necessary (order should be x,y,z) 

In [ ]:
Nx,Ny,Nz = stack.shape
print(stack.shape)
stack = np.transpose(stack, (1,2,0))

# 3D stack Preprocess stack

In [ ]:
if Nz == 8:                                 # i.e. multiplane data: remove the coregistration mask
    stack=cropCoregMask(stack)
    TEMP_NPIX = 100                         #  variable was undefined in mat file, set accordingly         
    stack=cropXY(stack,TEMP_NPIX - 4)       #  extra safety crop 
else:
    stack=cropXY(stack)
Nz,Nx,Ny = stack.shape
print(stack.shape)

In [20]:
# Phase retrieval
# define optics and processing parameters
s=phase_structure()
s.optics_kzT = 0.01                 # Axial cutoff frequency
# if set to [], use the theoretical value
s.proc_mirrorX = False              # mirror the input stack along X 
s.proc_mirrorZ = True               # mirror the input stack along Z
s.proc_applyFourierMask = True
# set experimental parameters
if stack.shape[2] == 8:              # i.e. MultiPlane data
    s.optics_dz = 0.35               # [um]
else:
    s.optics_dz = 0.2               # typical sampling for fixed cells

phase_structure.summarise(s)

Phase structure: 
_________________
s.optics_dx = 0.1083 	 	 s.optics_wv = 0.58
s.optics_dz = 0.2 	 	 s.optics_dlambda = 0.075
s.optics_NA = 1.3 	 	 s.optics_alpha = 4.21
s.optics_NA_ill = 0.19 	 	 s.optics_kzT = 0.01
s.optics_n = 1.406
Processing paramters: 
_________________
s.proc_mirrorX = False 	 	 s.proc_mirrorZ = True
s.proc_applyFourierMask = True


# compute the phase

In [21]:
QP,mask = getQP(stack,s)

Shifting FFT..
Inverse FFT..


In [22]:
QP.shape

(469, 469, 251)

In [23]:
#%% napari 3D stack plotting
if PLOT_FLAG:
    figID = os.path.basename(fname[0])
    plotStack(QP, figID)

# display phase and input stack next to each other

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0):
    c = 1
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(stack[:, :, z], cmap='gray')
    plt.title("input stack")

    plt.subplot(122)
    plt.imshow(QP[:, :, z], cmap='gray')
    plt.title("QP")
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, z= widgets.IntSlider(value=1, min=0, max=stack.shape[2]-1, step=1, description="Select Z", continuous_update=True))

interactive(children=(IntSlider(value=1, description='Select Z', max=250), Output()), _dom_classes=('widget-in…

<function __main__.update(z=0)>

In [25]:
def writeData(QP, path, filename, s): 
    outputImageFileName = os.path.join(path, f"QP_{filename}")    
    if len(QP.shape) == 4:
        QP = np.transpose(QP, (2,1,0,3))
        print("4d")
        axes = 'ZYXT'
    else: 
        QP = np.transpose(QP, (2,1,0))
        print("3d")
        axes = "ZXY"
    print(QP.shape)
    tf.imwrite(
        outputImageFileName,
        QP,
        resolution=(1/0.1083, 1/0.1083),
        metadata={ 
            'spacing': s.optics_dz,
            'unit': 'um',
            'finterval': 1,
            'axes': axes
        })

    print(f"writing QP stack {outputImageFileName} finished.")

In [26]:
path_out = os.path.join(path, "QP")
if not os.path.exists(path_out): #create output folder 
    os.mkdir(path_out)
    print("Created %s " % path_out)
    
separator = "\\"    
parse = fname[idx].split(separator)
file_out = parse[-1]
writeData(QP, path_out, file_out, s)

3d
(251, 469, 469)
writing QP stack C:\Users\mengelhardt\data\local\20220922\phase_calibration\1um_FOV1\stack_2\cropped\crop\QP\QP_stack2_l_3.tif finished.
